In [ ]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)


In [9]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
            deployment_name=AZURE_OPENAI_GPT4o_DEPLOYMENT,
            model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
            openai_api_version=AZURE_OPENAI_API_VERSION,
            api_key= AZURE_OPENAI_KEY,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            temperature=0,
            max_tokens=2000
            )




In [11]:
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

fileName = "./data/Best-Practices-in-Change-Management-Full-Report-Digital-11th-Edition.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

Number of pages:  367


![Upload](./images/map_reduce.png)

In [12]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [13]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [14]:
# This might take some time to run. For demo purposes, limit it to 10 pages.
map_reduce_outputs = map_reduce_chain.invoke({"input_documents": pages[100:110]})
print("Number of outputs: ", len(map_reduce_outputs))

Number of outputs:  3


In [15]:
display(Markdown(map_reduce_outputs['output_text']))

- **Increased Realization of Project Benefits**: Improved project performance, meeting objectives, sustaining benefits, faster delivery, and successful execution.
- **Improved Adoption of the Change**: Enhanced employee adoption, acceptance, compliance, speed of adoption, ultimate use, and proficiency.
- **Increased Engagement and Involvement**: Higher employee buy-in, engagement, participation, and confidence in the change.
- **Measurement of Outcomes**:
  - **Impacted Employee Responses**: Adoption, uptake, use, compliance metrics, surveys, and feedback.
  - **Project Results**: Meeting objectives, benefit realization, KPIs, ROI, revenue, cost, customer satisfaction, quality improvement.
  - **Project Execution**: Smooth implementation, timely deliverables, minimal disruption, productivity, and resistance.
- **Methods of Measuring Change Management (2015 & 2019)**:
  - Surveys, adoption/observation, feedback, assessments, eliminated challenges.
  - Channels: employee surveys, anecdotal evidence, presentations, industry data, lessons-learned reports.
- **Best Practices in Change Management**:
  - **Timing of Measurements**: Key milestones, continuous, pre and post-implementation.
  - **Individual Transition Measures**: Compliance, usage, attendance, engagement, performance evaluations.
  - **Data Sources**: End users, stakeholders, project teams, change agents, senior leaders.
  - **Meetings**: One-on-one and group meetings for performance evaluation.
- **Methods for Measuring Individual Progress**:
  - Observation, communication, tests, assessments, performance evaluations, existing business metrics, error rates, surveys.
- **2015 Study on Change Management Performance**:
  - Metrics: speed of adoption, ultimate utilization, proficiency.
  - Performance: 31% met expectations for speed, 39% for utilization, 36% for proficiency.
- **Measurement of Speed of Adoption (2016)**:
  - Metrics: project timeline completion, usage of new tools, surveys, assessments.
  - Effective programs correlated with better performance.
- **Methods and Metrics for Measuring Effectiveness**:
  - Feedback, observation, work output, timeline completion, proficiency, ultimate utilization.
- **Project Goals and KPIs**:
  - Adherence to schedule, budget, intended outcomes, employee usage, error rates, feedback, compliance.
- **Obstacles in Change Management Projects (2009)**:
  - **Leadership and Sponsorship**: Insufficient resources, lack of visibility, inconsistent leadership, slow decision-making, competing initiatives.
  - **Project Management**: Scope creep, unrealistic schedules, poor vendor management, insufficient resources, lack of business case.
  - **Change Management**: Employee resistance, insufficient resources, poor communication, inadequate training.
- **Best Practices for Early Change Management**:
  - Early initiation and structured methodology correlate with project success.

In [8]:
import pandas as pd
from pathlib import Path as p
final_mp_data = []
for doc, out in zip(map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,Best-Practices-in-Change-Management-Full-Repor...,.pdf,95,Prosci ® Best Practices in Change Managemen...,The main points of this text are:\n\n- Partici...
1,Best-Practices-in-Change-Management-Full-Repor...,.pdf,96,96 • © 2020 Prosci Inc. Figure 5.9 – Measure...,This chunk of text discusses the methods used ...
2,Best-Practices-in-Change-Management-Full-Repor...,.pdf,97,Prosci ® Best Practices in Change Managemen...,This text discusses the best practices in chan...
3,Best-Practices-in-Change-Management-Full-Repor...,.pdf,98,98 • © 2020 Prosci Inc. 2. Observation and c...,This chunk of text discusses various methods f...
4,Best-Practices-in-Change-Management-Full-Repor...,.pdf,99,Prosci ® Best Practices in Change Managemen...,This text discusses the findings of a study on...


In [9]:
index = 6
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Prosci ®    Best Practices in Change Management© 2020 Prosci Inc.  •  1013. Feedback and observation  
Participants collected feedback through interviews, 
focus groups and one-on-one conversations. 
Feedback was collected from key individuals 
including employees, managers, clients and change 
network members. Participants also gauged ultimate 
use by observing and monitoring employees in the 
new state.
4. Work output and performance Identified less than half as often as the previous 
finding, employee performance and output also 
determined ultimate use. Performance evaluations, 
project goals and updated key performance 
indicators (KPIs) were used. The quality and quantity 
of work output was also evaluated to determine how 
many employees were using the solution.
5. Timeline and process completion Participants tracked timeline, process and task 
completion often using checklists. Some said the time 
it took for employees to become efficient using a new 
solution contrib